In [1]:
# default_exp database

# module name here

> API details.

In [2]:
#hide
import logging
logging.basicConfig(level= logging.INFO)
log = logging.getLogger("pynamodb")
log.setLevel(logging.DEBUG)
log.setLevel(logging.WARNING)
log.propagate = True


In [3]:
#hide
import pickle, os
KEY = ''
PW = ''
keypath = '/Users/nic/.villa-master-inventory-tester'
if KEY and PW:
  with open (keypath, 'wb') as f:
    pickle.dump({
        'KEY': KEY,
        'PW': PW
    }, f)
with open(keypath, 'rb') as f:
  creden = pickle.load(f)
USER = creden['KEY']
PW = creden['PW']
ACCESS_KEY_ID = USER
SECRET_ACCESS_KEY = PW
os.environ['DATABASE_TABLE_NAME'] = 'inventory-database-dev-manual'
os.environ['REGION'] = 'ap-southeast-1'
os.environ['INVENTORY_BUCKET_NAME'] = 'inventory-bucket-dev-manual'
os.environ['INPUT_BUCKET_NAME'] = 'input-bucket-dev-manual'
# os.environ['DAX_ENDPOINT'] = None
REGION = 'ap-southeast-1'

FileNotFoundError: [Errno 2] No such file or directory: '/Users/nic/.villa-master-inventory-tester'

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from villaInvDatabase.helper import DatabaseHelper
from villaInvDatabase.s3 import DatabaseS3
from villaInvDatabase.query import Querier
from villaInvDatabase.update import Updater
import pandas as pd
from datetime import datetime
from pynamodb.models import Model
from pynamodb.attributes import UnicodeAttribute, NumberAttribute, JSONAttribute, BooleanAttribute, BinaryAttribute
from pynamodb.indexes import GlobalSecondaryIndex, AllProjection
from botocore.config import Config
from s3bz.s3bz import S3
from pprint import pprint

import pickle, json, boto3, bz2, requests, validators, os, logging

In [ ]:
#export
try:
  DATABASE_TABLE_NAME = os.environ['DATABASE_TABLE_NAME']
  INVENTORY_BUCKET_NAME = os.environ['INVENTORY_BUCKET_NAME']
  INPUT_BUCKET_NAME = os.environ['INPUT_BUCKET_NAME']
  REGION = os.environ['REGION']
  ACCESS_KEY_ID = None
  SECRET_ACCESS_KEY = None
except Exception as e:
  print(f'error, missing environment variables \n{e}')
  DATABASE_TABLE_NAME = None
  INVENTORY_BUCKET_NAME = None
  INPUT_BUCKET_NAME = None
  REGION = 'ap-southeast-1'
try:
  DAX_ENDPOINT = os.environ['DAX_ENDPOINT']
except:
  DAX_ENDPOINT = None
  print('dax endpoint missing')


# Secondary Index class

In [ ]:
#export
class NeedUpdateIndex(GlobalSecondaryIndex):
  class Meta:
    index_name = 'need-update'
    projection = AllProjection()
    read_capacity_units = 1
    write_capacity_units = 1
  needUpdate = UnicodeAttribute(default=False, hash_key=True)

# Main Database Class

In [ ]:
#export
# dont forget to import dependent classes from the relevant notebooks
class Database(Model, DatabaseS3, Updater, Querier, DatabaseHelper):
  class Meta:
    aws_access_key_id = ACCESS_KEY_ID
    aws_secret_access_key = SECRET_ACCESS_KEY
    table_name = DATABASE_TABLE_NAME
    region = REGION
    billing_mode='PAY_PER_REQUEST'
    dax_read_endpoints = DAX_ENDPOINT
    dax_write_endpoints = DAX_ENDPOINT
    
  ib_prcode = UnicodeAttribute(hash_key=True, default = '')
  lastUpdate = NumberAttribute( default = 0)
  inventory = JSONAttribute(default={})
  needUpdate = UnicodeAttribute(default= 'N')
  needUpdateIndex = NeedUpdateIndex()
  TRUE = 'Y'
  FALSE = 'N'
  
  def __repr__(self):
    invDict = self.inventory
    invDict['ib_prcode'] = self.ib_prcode
    return json.dumps(invDict)
  def setNoUpdate(self, batch = None):
    self.needUpdate = self.FALSE
    if batch:
      return batch.save(self)
    else:
      return self.save()
  def setUpdate(self):
    self.needUpdate = self.TRUE
    return self.save()

# Tests

In [ ]:
#hide
# test saving to s3
S3.save(
   key='test',
   objectToSave = {'test':'test'},
   bucket = INPUT_BUCKET_NAME,
   user = USER,
   pw = PW,
   accelerate = True
)

print( S3.exist('test', INPUT_BUCKET_NAME, user=USER, pw=PW, accelerate=True))

S3.load(
  key = 'test',
  bucket = INPUT_BUCKET_NAME ,
  user = USER, 
  pw = PW
)

In [ ]:
#hide
# test saving item
item = Database(
  ib_prcode = '123test',
  lastUpdate = 1,
  inventory = {'test': '1'},
  needUpdate = 'N' )
item.save()

In [ ]:
sampleInput = [
               {
                  'ib_prcode': '0000009',
                  'ib_brcode': '1000',
                  'ib_cf_qty': '50',
                  'new_ib_vs_stock_cv': '27'
                },
               {
                  'ib_prcode': '0000002',
                  'ib_brcode': '1000',
                  'ib_cf_qty': '35',
                  'new_ib_vs_stock_cv': '33'
               }
              ]

## Dump database to s3

In [ ]:
Database.dumpToS3(user=USER, pw = PW)

In [ ]:
Database.splitBranches(bucket = INVENTORY_BUCKET_NAME, user=USER, pw=PW)

# Save using Standard

In [ ]:
Database.updateLambdaInput(sampleInput)

## Save using s3

In [ ]:
inputKeyName = 'input-data-name'
saveResult = S3.save(key=inputKeyName, 
                     objectToSave = sampleInput , 
                     bucket = INPUT_BUCKET_NAME,
                     user = USER,
                     pw = PW,
                     accelerate = False)
logging.info('test input data saved to s3')
updateResult = Database.updateS3Input(
  inputBucketName=INPUT_BUCKET_NAME, key= inputKeyName,
  user = USER, pw = PW)

logging.info(f's3 save result is {saveResult} update result is {updateResult}')

## Query test

### Product Query

In [ ]:
sampleQueryInput = {
    'ib_prcode': '0000002'
}  

In [ ]:
Database.singleProductQuery(sampleQueryInput)

### Branch Query

In [ ]:
from s3bz.s3bz import Requests
branchURL = Database.branchQuery('1000', bucket = INVENTORY_BUCKET_NAME, user=USER, pw=PW)
print(branchURL)
Requests.getContentFromUrl(branchURL)

### AllQuery

In [ ]:
from s3bz.s3bz import Requests
branchURL = Database.allQuery(bucket = INVENTORY_BUCKET_NAME, user=USER, pw=PW)
print(branchURL)
Requests.getContentFromUrl(branchURL)

In [ ]:
#hide
print('passed the unit test')